DEFINE "PATH_TO_IMAGES" on your local machine

In [1]:
import sys
import os

import pandas as pd

import torch
import torch.optim as optim
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau

import pprint

from warnings import filterwarnings
filterwarnings("ignore")

sys.path.append(os.path.join(str(os.path.abspath('')), "..", "..", "..", "..", ".."))

from src.classification.resNet_model import CovidResNetModel
from src.classification.efficientNet_model import CovidEffnetModel
from src.classification.generalized_clf import CovidGeneralModel
from src.classification.train_one_epoch import train_one_epoch
from src.classification.validate_func import valid_func
from src.classification.classification_transforms import get_transform
from src.classification.get_data_loaders import prepare_train_valid_dataloader

In [2]:
class CFG:
    debug = False # change this to run on full data
    
    image_size = 512
    target_columns = [
        'Negative for Pneumonia',
        'Typical Appearance',
        'Indeterminate Appearance',
        'Atypical Appearance'
    ]
    folds = [0, 1, 2, 3, 4]
    
    kernel_type = 'tf_efficientnet_b4_ns'
    train_batch_size = 16
    valid_batch_size = 24
    
    init_lr = 1e-3
    weight_decay=0
    
    n_epochs = 20
    num_workers = 4

    use_amp=False
    early_stop = 5

    data_dir = PATH_TO_IMAGES # !!! DEFINE "PATH_TO_IMAGES" on your local machine
    model_dir = f'weights/'
    seed=42
    

In [3]:
! mkdir $CFG.model_dir

In [4]:
df_train = pd.read_csv('../cross_validation_scheme/crossval_info/original_data/multi_label_problem.csv')
df_train['image_name'] = df_train['image_id'].apply(lambda x: x.replace('_image', '.png'))
df_train['file_path'] = df_train.image_name.apply(lambda x: os.path.join(CFG.data_dir, f'{x}'))
if CFG.debug:
    df_train = df_train.sample(frac=0.1)

In [5]:
train_transform = get_transform(img_size=CFG.image_size, transform_type='try_new_strong')
valid_transform = get_transform(img_size=CFG.image_size, transform_type='val')

criterion = nn.BCEWithLogitsLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
for fold in CFG.folds:
    train_loader, valid_loader = prepare_train_valid_dataloader(
        df=df_train, fold=fold, target_cols=CFG.target_columns,
        train_transform=train_transform, valid_transform=valid_transform,
        train_batch_size=CFG.train_batch_size, valid_batch_size=CFG.valid_batch_size,
        num_workers=CFG.num_workers
    )
    
    model = CovidEffnetModel(model_name=CFG.kernel_type, pretrained=True).to(device)
#     model = CovidGeneralModel(model_name=CFG.kernel_type, pretrained=True).to(device)
    optimizer = optim.Adam(model.parameters(), lr=CFG.init_lr, weight_decay=CFG.weight_decay)
    scheduler = ReduceLROnPlateau(
        optimizer, mode='min', patience=1, min_lr=1e-6, factor=0.1, verbose=True, eps=1e-8
    )

    num_epochs = CFG.n_epochs

    print("-----------------------------------------------------------------------------------------------------")
    print("                                        FOLD: ", fold)
    print("-----------------------------------------------------------------------------------------------------")
    
    roc_auc_max = 0.0
    loss_min = 99999
    ap_max = 0.0
    not_improving = 0
    metrics_list = list()
    
    for epoch in range(num_epochs):
        
        loss_train, roc_auc_train, average_precision_train = train_one_epoch(
            model, device, criterion, optimizer, train_loader, CFG.use_amp, CFG.target_columns)
        
        loss_valid, roc_auc_valid, average_precision_valid = valid_func(
            model, device, criterion, valid_loader, CFG.target_columns)
        
        scheduler.step(loss_valid)
        
        metrics_dictionary = {}
        metrics_dictionary['epoch'] = epoch
        metrics_dictionary['loss_train'] = loss_train
        metrics_dictionary['loss_valid'] = loss_valid
        metrics_dictionary['roc_auc_train'] = roc_auc_train
        metrics_dictionary['roc_auc_valid'] = roc_auc_valid
        metrics_dictionary['average_precision_train'] = average_precision_train
        metrics_dictionary['average_precision_valid'] = average_precision_valid
        metrics_dictionary['fold'] = fold
        pprint.pprint(metrics_dictionary)
        metrics_list.append(metrics_dictionary)
        
        not_improving += 1
        if roc_auc_valid > roc_auc_max:
            print(f'roc_auc_max ({roc_auc_max:.6f} --> {roc_auc_valid:.6f}). Saving model ...')
            torch.save(model.state_dict(), f'{CFG.model_dir}{CFG.kernel_type}_fold{fold}_best_AUC.pth')
            roc_auc_max = roc_auc_valid
            not_improving = 0

        if loss_valid < loss_min:
            loss_min = loss_valid
            torch.save(model.state_dict(), f'{CFG.model_dir}{CFG.kernel_type}_fold{fold}_best_loss.pth')

        if average_precision_valid > ap_max:
            print(f'ap_max ({ap_max:.6f} --> {average_precision_valid:.6f}). Saving model ...')
            torch.save(model.state_dict(), f'{CFG.model_dir}{CFG.kernel_type}_fold{fold}_best_AP.pth')
            ap_max = average_precision_valid
#             not_improving = 0
            
        if not_improving == CFG.early_stop:
            print('Early Stopping...')
            break

    
    metrics = pd.DataFrame(metrics_list)
    metrics.to_csv(f'{CFG.model_dir}{CFG.kernel_type}_fold{fold}_final.csv', index=False)
    torch.save(model.state_dict(), f'{CFG.model_dir}{CFG.kernel_type}_fold{fold}_final.pth')


  0%|          | 0/317 [00:00<?, ?it/s]

-----------------------------------------------------------------------------------------------------
                                        FOLD:  0
-----------------------------------------------------------------------------------------------------


loss: 0.42737, smth: 0.40772: 100%|██████████| 53/53 [00:07<00:00,  6.71it/s]


{'average_precision_train': 0.25716924236243693,
 'average_precision_valid': 0.30586909179275185,
 'epoch': 0,
 'fold': 0,
 'loss_train': 0.4590297140157561,
 'loss_valid': 0.404657281794638,
 'roc_auc_train': 0.6542507489913345,
 'roc_auc_valid': 0.7424658334550189}
roc_auc_max (0.000000 --> 0.742466). Saving model ...
ap_max (0.000000 --> 0.305869). Saving model ...


  0%|          | 0/317 [00:00<?, ?it/s]

...

{'average_precision_train': 0.3826769418809473,
 'average_precision_valid': 0.36752433163528325,
 'epoch': 13,
 'fold': 4,
 'loss_train': 0.35158048908605183,
 'loss_valid': 0.3745445600658093,
 'roc_auc_train': 0.8242396756678594,
 'roc_auc_valid': 0.8067322799649788}


  0%|          | 0/317 [00:00<?, ?it/s]

Epoch    15: reducing learning rate of group 0 to 1.0000e-06.
{'average_precision_train': 0.3807739515983335,
 'average_precision_valid': 0.3682032912673956,
 'epoch': 14,
 'fold': 4,
 'loss_train': 0.3525073706915702,
 'loss_valid': 0.3728827862244732,
 'roc_auc_train': 0.8193998456545268,
 'roc_auc_valid': 0.8080411439762625}


loss: 0.57435, smth: 0.38729: 100%|██████████| 53/53 [00:07<00:00,  7.08it/s]


{'average_precision_train': 0.38591187395746274,
 'average_precision_valid': 0.36838568710163727,
 'epoch': 15,
 'fold': 4,
 'loss_train': 0.3503647302791523,
 'loss_valid': 0.3736475374338762,
 'roc_auc_train': 0.8238557865560044,
 'roc_auc_valid': 0.8069385339305489}
Early Stopping...
